### Etapa 1 - Preparando o Ambiente

##### Criando um ambiente virtual e o ativando (no terminal)

python -m venv venv
venv\Scripts\activate

#### Instalando e importando as bibliotecas necessarias (no terminal)

##### Instalando (terminal)

pip install pyspark


##### Importando


In [38]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import lit, when, col, expr, udf, rand, format_number
from pyspark.sql.types import StringType
import random
import os
import pandas as pd


#### Configurando o Spark Session

In [39]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Exercício Intro") \
    .getOrCreate()

#### Carregar Arquivo nomes_aleatorios.txt, adicionando uma coluna chamada "Nome" e exibindo as primeiras 5 linhas

In [1]:
caminho_arquivo = "../exerciciogerardados/nomes_aleatorios.txt"

df_nomes = spark.read.csv(caminho_arquivo, header=False, inferSchema=True) \
    .toDF("Nome")  

df_nomes.show(5)


NameError: name 'spark' is not defined

Rsultado:
<p>

![etapa1](../../Evidencias/exercicioapachespark/carregaarquivonome.png)

### Etapa 2: Renomeando Coluna e Mostrando Schema

In [41]:
# Renomeando a coluna
from pyspark.sql.functions import col

df_nomes = df_nomes.withColumnRenamed("nome", "Nomes")

# Mostrando o Schema e 10 linhas
df_nomes.printSchema()

df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



Resultado:

<p>

![etapa2](../../Evidencias/exercicioapachespark/renomeandocolunaemostrandoschema.png)

### Etapa 3: Adicionando a Coluna Escolaridade 

In [42]:
# Lista de valores possíveis
escolaridade = ["Fundamental", "Médio", "Superior"]

# Função UDF para gerar valores aleatórios
def gerar_escolaridade():
    return random.choice(escolaridade)

# Registrando UDF
escolaridade_udf = udf(gerar_escolaridade, StringType())

# Adicionando coluna
df_nomes = df_nomes.withColumn("Escolaridade", escolaridade_udf())
df_nomes.show(5)

+----------------+------------+
|           Nomes|Escolaridade|
+----------------+------------+
|  Frances Bennet|    Superior|
|   Jamie Russell|       Médio|
|  Edward Kistler| Fundamental|
|   Sheila Maurer|       Médio|
|Donald Golightly| Fundamental|
+----------------+------------+
only showing top 5 rows



Resultado: 

<p>

![etapa3](../../Evidencias/exercicioapachespark/escolaridade.png)

### Etapa 4: Adicionando a Coluna País 

In [43]:
paises = ["Argentina", "Bolívia", "Brasil", "Chile", "Colômbia", "Equador", "Guiana", "Paraguai", "Peru", "Suriname", "Uruguai", "Venezuela", "Guiana Francesa"]

# Gerando a coluna de maneira aleatoria

df_nomes = df_nomes.withColumn("País", expr(f"element_at(array({', '.join([f'\'{p}\'' for p in paises])}), int(rand() * {len(paises)} + 1))"))
df_nomes.show(5)

+----------------+------------+---------------+
|           Nomes|Escolaridade|           País|
+----------------+------------+---------------+
|  Frances Bennet| Fundamental|      Venezuela|
|   Jamie Russell| Fundamental|         Guiana|
|  Edward Kistler|       Médio|       Paraguai|
|   Sheila Maurer|       Médio|Guiana Francesa|
|Donald Golightly|    Superior|Guiana Francesa|
+----------------+------------+---------------+
only showing top 5 rows



Resultado:

<p>

![etapa4](../../Evidencias/exercicioapachespark/addcolunapais.png)

### Etapa 5: Adicionar Coluna AnoNascimento 

In [44]:
# Adicionando a coluna AnoNascimento
df_nomes = df_nomes.withColumn("AnoNascimento", expr("cast(1945 + floor(rand() * (2010 - 1945 + 1)) as int)"))
df_nomes.show(5)

+----------------+------------+---------------+-------------+
|           Nomes|Escolaridade|           País|AnoNascimento|
+----------------+------------+---------------+-------------+
|  Frances Bennet| Fundamental|      Venezuela|         1966|
|   Jamie Russell|       Médio|         Guiana|         1968|
|  Edward Kistler| Fundamental|       Paraguai|         1988|
|   Sheila Maurer|    Superior|Guiana Francesa|         1957|
|Donald Golightly| Fundamental|Guiana Francesa|         1972|
+----------------+------------+---------------+-------------+
only showing top 5 rows



Resultado:

<p>

![etapa5](../../Evidencias/exercicioapachespark/addcolunaanonasc.png)

### Etapa 6: Selecionando as Pessoas Nascidas no Século 21

In [45]:
df_select = df_nomes.filter(col("AnoNascimento") >= 2000)
df_select.show(10)

+---------------+------------+--------+-------------+
|          Nomes|Escolaridade|    País|AnoNascimento|
+---------------+------------+--------+-------------+
|  Tracy Herring| Fundamental|    Peru|         2005|
|   Leroy Strahl| Fundamental|Suriname|         2007|
|    Albert Leef|    Superior|   Chile|         2005|
|Helen Blackwell| Fundamental|Paraguai|         2002|
| Gabriel Colyer| Fundamental|Suriname|         2005|
|       Mary Lee|       Médio|   Chile|         2009|
|     Anita Ross| Fundamental| Equador|         2000|
|    Lisa Baxley|       Médio|  Guiana|         2010|
|   Juliet Liles| Fundamental|  Brasil|         2007|
|      Ana Baker|       Médio|Paraguai|         2010|
+---------------+------------+--------+-------------+
only showing top 10 rows



Resultado:

<p>

![etapa6](../../Evidencias/exercicioapachespark/selecionandopessoassec21.png)

### Etapa 7: Seleção Utilizando Spark SQL

In [46]:
df_nomes.createOrReplaceTempView("pessoas")
df_select_sql = spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000")
df_select_sql.show(10)

+---------------+------------+--------+-------------+
|          Nomes|Escolaridade|    País|AnoNascimento|
+---------------+------------+--------+-------------+
|  Tracy Herring|    Superior|    Peru|         2005|
|   Leroy Strahl|    Superior|Suriname|         2007|
|    Albert Leef|    Superior|   Chile|         2005|
|Helen Blackwell| Fundamental|Paraguai|         2002|
| Gabriel Colyer|    Superior|Suriname|         2005|
|       Mary Lee|       Médio|   Chile|         2009|
|     Anita Ross|    Superior| Equador|         2000|
|    Lisa Baxley| Fundamental|  Guiana|         2010|
|   Juliet Liles| Fundamental|  Brasil|         2007|
|      Ana Baker|       Médio|Paraguai|         2010|
+---------------+------------+--------+-------------+
only showing top 10 rows



![etapa7](../../Evidencias/exercicioapachespark/requesql.png)

### Etapa 8: Contando Millennials com Filter

In [47]:
millennials_count = df_nomes.filter((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994)).count()

print(f"Millennials: {millennials_count:,}")


Millennials: 226,922


Resultado:

<p>

![etapa8](../../Evidencias/exercicioapachespark/countmillenials.png)

### Etapa 9: Contando os Millennials com Spark SQL

In [48]:
millennials_count_sql = spark.sql("SELECT COUNT(*) as `Contador dos Millenials` FROM pessoas WHERE AnoNascimento >= 1980 AND AnoNascimento <= 1994")

# Formatando o número
millennials_count_sql = millennials_count_sql.select(format_number("`Contador dos Millenials`", 0).alias("Contador dos Millenials"))
millennials_count_sql.show()


+-----------------------+
|Contador dos Millenials|
+-----------------------+
|                226,922|
+-----------------------+



![etapa9](../../Evidencias/exercicioapachespark/ccontandocmsql.png)

### Etapa 10: Distribuição Populacional por País e Geração



In [49]:
# Adicionando a coluna Geração
df_nomes = df_nomes.withColumn("Geração", 
    when((col("AnoNascimento") >= 1944) & (col("AnoNascimento") <= 1964), "Baby Boomers")
    .when((col("AnoNascimento") >= 1965) & (col("AnoNascimento") <= 1979), "Geração X")
    .when((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994), "Millennials")
    .when((col("AnoNascimento") >= 1995) & (col("AnoNascimento") <= 2015), "Geração Z")
)
# Contagem por País e Geração
df_contagem = df_nomes.groupBy("País", "Geração").count().orderBy("País", "Geração")

# Formatar a coluna de contagem com vírgulas
df_contagem_formatado = df_contagem.select(
    "País", 
    "Geração", 
    format_number("count", 0).alias("Contagem")
)

# Convertendo para Pandas e exibindo
df_contagem_pd = df_contagem_formatado.toPandas()
display(df_contagem_pd)


,País,Geração,Contagem
0,Argentina,Baby Boomers,"23,574"
1,Argentina,Geração X,"17,633"
2,Argentina,Geração Z,"18,655"
3,Argentina,Millennials,"17,513"
4,Bolívia,Baby Boomers,"23,194"
5,Bolívia,Geração X,"17,519"
6,Bolívia,Geração Z,"18,640"
7,Bolívia,Millennials,"17,413"
8,Brasil,Baby Boomers,"22,868"
9,Brasil,Geração X,"17,444"


Resultado:

<p>

![etapa10pt1](../../Evidencias/exercicioapachespark/distpop1.png)

![etapa10pt2](../../Evidencias/exercicioapachespark/distpop2.png)

![etapa10pt3](../../Evidencias/exercicioapachespark/distpop3.png)